In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16895,2024-03-02,Espanha Acb,14:00,Manresa,Murcia,1.69,2.19,167.5,1.85,1.95,-3.5,2.01,2.57,EZz0VDrG,0.591716,0.456621,0.540541,0.512821,0.048337,0.4,0.8,NaN,NaN,174.936,43.807098,0.250418,195.716,75.251978,0.384496,166.062,182.966,233.31,238.71,0.0,0.0,0.0,0.0,0.182244,0.037216,0.172917,-1.93,-0.386,-1.787565,0.000000,0.0,0.000000,-1.75,-0.350,-3.400000,0.000000,0.0,0.000000
16896,2024-03-02,Eua Nba,00:00,Chicago Bulls,Milwaukee Bucks,2.56,1.55,221.5,1.85,2.09,4.5,1.97,1.27,UuYtAFt8,0.390625,0.645161,0.540541,0.478469,0.035786,0.8,0.0,NaN,NaN,364.948,70.726424,0.193799,185.992,67.968895,0.365440,287.702,208.050,331.32,165.41,1.0,0.0,0.0,0.0,0.347532,0.086145,0.305540,1.36,0.272,5.735294,0.547050,0.6,0.052950,-1.09,-0.218,-2.522936,0.655623,0.5,-0.155623
16897,2024-03-02,Eua Nba,17:00,Brooklyn Nets,Atlanta Hawks,1.73,2.18,220.5,1.81,1.95,-3.5,2.04,2.61,GdWl8gBK,0.578035,0.458716,0.552486,0.512821,0.036750,0.2,0.4,NaN,NaN,319.754,181.135190,0.566483,185.102,31.289104,0.169037,262.274,322.308,433.40,130.68,1.0,0.0,0.0,0.0,0.162761,0.052657,0.173355,-3.61,-0.722,-1.011080,0.488121,0.3,-0.188121,-3.49,-0.698,-1.690544,0.463850,0.3,-0.163850
16898,2024-03-02,Alemanha Bbl,14:30,Syntainics MBC,Rostock,1.77,2.05,178.5,1.82,1.94,-2.5,2.03,2.36,OIsidrlq,0.564972,0.487805,0.549451,0.515464,0.052777,1.0,0.6,NaN,NaN,321.664,151.123396,0.469818,230.988,78.017996,0.337758,305.136,326.112,589.10,254.54,1.0,0.0,0.0,0.0,0.103660,0.045134,0.106308,-3.40,-0.680,-1.132353,0.454487,0.2,-0.254487,1.40,0.280,3.750000,0.000000,0.0,0.000000
16899,2024-03-02,Alemanha Bbl,16:00,Vechta,Ludwigsburg,1.69,2.19,171.5,1.86,1.94,-3.5,2.00,2.45,xMoee23k,0.591716,0.456621,0.537634,0.515464,0.048337,0.4,0.4,NaN,NaN,147.926,41.029027,0.277362,127.794,11.753888,0.091975,146.228,133.700,201.14,128.00,0.0,0.0,0.0,0.0,0.182244,0.029773,0.143010,-1.22,-0.244,-2.827869,0.000000,0.0,0.000000,-2.42,-0.484,-2.458678,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16957,2024-03-02,Eua Ncaa,22:30,Southern Utah,Tarleton,2.18,1.73,150.5,1.91,1.91,1.5,2.00,1.56,l4ZsduCJ,0.458716,0.578035,0.523560,0.523560,0.036750,0.2,0.8,NaN,NaN,185.992,108.501375,0.583366,202.552,61.826661,0.305238,128.362,187.358,113.56,197.60,0.0,0.0,0.0,0.0,0.162761,0.000000,0.174790,-3.81,-0.762,-1.548556,0.528963,0.4,-0.128963,7.00,1.400,0.521429,0.334133,0.7,0.365867
16958,2024-03-02,Eua Ncaa,23:00,CS Fullerton,UC Santa Barbara,1.95,1.87,139.5,1.91,1.91,-1.5,2.05,1.95,lQAI8IeP,0.512821,0.534759,0.523560,0.523560,0.047580,0.4,0.6,NaN,NaN,168.354,117.177539,0.696019,172.116,94.751918,0.550512,111.296,198.266,103.80,338.79,0.0,0.0,0.0,0.0,0.029617,0.000000,0.035355,-2.26,-0.452,-2.101770,0.000000,0.0,0.000000,1.33,0.266,3.270677,0.422839,0.4,-0.022839
16959,2024-03-02,Eua Ncaa,23:00,Utah Tech,Abilene Christian,1.91,1.91,147.5,1.80,1.95,-1.5,1.95,2.05,z1o0XeZI,0.523560,0.523560,0.555556,0.512821,0.047120,0.4,0.4,NaN,NaN,187.568,65.527421,0.349353,167.546,53.135412,0.317139,176.034,179.332,205.70,175.56,0.0,0.0,0.0,0.0,0.000000,0.056569,0.035355,3.85,0.770,1.181818,0.000000,0.0,0.000000,-0.99,-0.198,-4.595960,0.342640,0.3,-0.042640
16960,2024-03-02,México Cibacopa,23:00,Venados de Mazatlan,Caballeros de Culiacan,1.95,1.72,177.5,1.83,1.83,1.5,1.83,1.63,2JgQ

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Espanha Acb,Manresa,Murcia,167.5,1.85,1.0000,Over
1,17:00,Eua Nba,Brooklyn Nets,Atlanta Hawks,220.5,1.81,1.0000,Over
2,14:30,Alemanha Bbl,Syntainics MBC,Rostock,178.5,1.82,1.0000,Over
3,16:00,Alemanha Bbl,Vechta,Ludwigsburg,171.5,1.86,1.0000,Over
4,13:30,Áustria Superliga,St. Polten,UBSC Graz,153.5,1.86,1.0000,Over
5,02:00,Coreia Do Sul Kbl,KCC Egis,Suwon KT,169.5,1.89,1.0000,Over
6,17:00,Espanha Leb Plata,Gran Canaria B,Basquet L'Horta Godella,146.5,1.85,1.0000,Over
7,13:00,Finlândia Korisliiga,UU-Korihait,BC Nokia,171.5,1.86,1.0000,Over
8,13:45,Grécia Liga De Basquete,PAOK,AEK Athens,158.5,1.88,1.0000,Over
9,14:00,Hungria Nacional,Zalakeramia,Soproni KC,167.5,1.85,1.0000,Over
